In [1]:
import pandas as pd

In [2]:
# Load the CSV file into a DataFrame
prices = pd.read_csv('agile-half-hour-actual-rates-01-01-2023_31-12-2023.csv')

# Drop the 'Period to' column
prices.drop(columns=["Period to"], inplace=True)

# Convert the 'Period from' column to datetime format
prices['Period from'] = pd.to_datetime(prices['Period from'], format='%d/%m/%Y %H:%M')

# Check for missing dates
missing_dates = pd.date_range(start='2023-01-01 00:00', end='2023-12-31 23:30', freq='30min').difference(prices['Period from'])
print("Missing dates:", missing_dates)

Missing dates: DatetimeIndex(['2023-03-26 01:00:00', '2023-03-26 01:30:00'], dtype='datetime64[ns]', freq='30min')


In [3]:
# Set the 'Period from' column as the index
prices.set_index('Period from', inplace=True)

In [4]:
avg_import_price = (prices.loc[missing_dates[0] - pd.Timedelta(minutes=30), 'Agile Import price (p/kWh)'] + prices.loc[missing_dates[-1] + pd.Timedelta(minutes=30), 'Agile Import price (p/kWh)']) / 2
avg_export_price = (prices.loc[missing_dates[0] - pd.Timedelta(minutes=30), 'Agile Export price (p/kWh)'] + prices.loc[missing_dates[-1] + pd.Timedelta(minutes=30), 'Agile Export price (p/kWh)']) / 2

In [5]:
# Create a new row with the average values
new_rows = pd.DataFrame({
    'Agile Import price (p/kWh)': [avg_import_price, avg_import_price],
    'Agile Export price (p/kWh)': [avg_export_price, avg_export_price],
}, index=[missing_dates[0], missing_dates[-1]])

# Append the new rows to the DataFrame
prices = pd.concat([prices, new_rows])

# Sort the DataFrame by the index (Date)
prices.sort_index(inplace=True)

In [6]:
prices.loc[missing_dates[0] - pd.Timedelta(minutes=60):missing_dates[-1] + pd.Timedelta(minutes=60)]

,Agile Import price (p/kWh),Agile Export price (p/kWh)
2023-03-26 00:00:00,26.80,12.41
2023-03-26 00:30:00,28.66,13.22
2023-03-26 01:00:00,27.23,12.60
2023-03-26 01:30:00,27.23,12.60
2023-03-26 02:00:00,25.80,11.98
2023-03-26 02:30:00,24.92,11.60


In [7]:
# Reset the index
prices.reset_index(inplace=True)
prices.rename(columns={'index': 'Date'}, inplace=True)

In [8]:
simulated_data = pd.read_csv("./Simulation Results/simulation_results_concatenated.csv")

In [9]:
# Concatenate DataFrames horizontally
results = pd.concat([simulated_data, prices], axis=1)

In [10]:
results.drop(columns=["DateTime"], inplace=True)

In [11]:
# Set the 'Date' column as the index
results.set_index('Date', inplace=True)

In [12]:
results.head()

,Dry-bulb temperature (°C),Total system energy (kBtu/h),System natural gas (kBtu/h),System electricity (kBtu/h),Total lights energy (kBtu/h),Lights electricity (kBtu/h),Total equip energy (kBtu/h),Equip electricity (kBtu/h),Total CE ex equip (kgCO2/h),Total system CE (kgCO2/h),...,Electricity (mu),Gas (mu),Renewables (mu),Total nat. gas (kBtu/h),Total electricity (kBtu/h),Total grid displaced elec. PV (kBtu/h),Total energy (kBtu/h),Transformer Losses (kBtu/h),Agile Import price (p/kWh),Agile Export price (p/kWh)
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,5.5,7.460,7.325,0.135,0.0,0.0,2.912,2.912,0.5,0.5,...,0.06,0.03,0.0,7.325,3.048,0.0,10.372,0.0,4.41,2.77
2023-01-01 00:30:00,5.5,7.465,7.330,0.135,0.0,0.0,2.912,2.912,0.5,0.5,...,0.06,0.03,0.0,7.330,3.048,0.0,10.378,0.0,13.91,6.86
2023-01-01 01:00:00,5.5,7.468,7.333,0.135,0.0,0.0,2.912,2.912,0.5,0.5,...,0.06,0.03,0.0,7.333,3.048,0.0,10.380,0.0,15.44,7.52
2023-01-01 01:30:00,5.5,7.467,7.331,0.135,0.0,0.0,2.912,2.912,0.5,0.5,...,0.06,0.03,0.0,7.331,3.048,0.0,10.379,0.0,1.10,1.34
2023-01-01 02:00:00,5.5,7.465,7.330,0.135,0.0,0.0,2.912,2.912,0.5,0.5,...,0.06,0.03,0.0,7.330,3.048,0.0,10.378,0.0,6.70,3.76


In [13]:
# Export the DataFrame to a CSV file
# results.to_csv('simulation_results_completed.csv', index=True)

In [14]:
# Slice the DataFrame from '15/01/2023 00:00' to the end
start_date = '2023-01-15 00:00:00'
filtered_results = results.loc[start_date:]